# Load packages

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import yfinance as yf
import seaborn as sns

from matplotlib import pyplot as plt

# Configuration

In [ ]:
# Korean font
from matplotlib import font_manager, rc
try:
    font_path = "C:/Windows/Fonts/malgun.TTF"
    Kfont = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=Kfont)
except:
    pass

# Fix minus presentation
mpl.rcParams['axes.unicode_minus'] = False

# Load data

In [ ]:
# Load several index information
snp500 = yf.Ticker('^GSPC')
snp500.history(period='max')